Using OAI Function Calling

* https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=python

In [6]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../creds/.env")

True

In [7]:
from openai import AzureOpenAI

azure_endpoint = "https://cursor-gpt-4.openai.azure.com"
api_version="2024-02-15-preview"


client = AzureOpenAI(
        azure_endpoint=azure_endpoint,
        api_version=api_version,
    )

In [5]:
# THREE types of tools: Code Interpreter, Retrieval, Function Calling
import os
import json

# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [13]:
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    # use system prompt
    messages = [{"role": "system", "content": "You're an AI assistant designed to help users return current weather. When a user asks for the weather in a given location, you should call the get_current_weather function. Don't make assumptions about what values to use with functions. Ask for clarification if a user request is ambiguous. Only use the functions you have been provided with."}, {"role": "user", "content": "who is Jeff Dean?"} # What's the weather like in San Francisco, Tokyo, and Paris?
]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            # mode details the better in desc
                            # "description": "The location of the hotel. The location should include the city and the state's abbreviation (i.e. Seattle, WA or Miami, FL)"
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    
    response = client.chat.completions.create(
    model="pjf-dpo-turbo-35",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, or none
    seed=42,
    temperature=0,
    max_tokens =200, 
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        print("tools called")
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="pjf-dpo-turbo-35",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
    else:
        return response_message
print(run_conversation())

ChatCompletionMessage(content="Jeff Dean is a prominent computer scientist and software engineer known for his work at Google. He has made significant contributions to the field of computer science, particularly in the areas of distributed systems, large-scale data processing, and machine learning. Jeff Dean is widely recognized for his role in developing and optimizing key technologies that power Google's infrastructure and services.", role='assistant', function_call=None, tool_calls=None)


More complex example, JSON extraction

In [1]:
student_custom_functions = [
  {
      'name': 'extract_student_info',
      'description': 'Get the student information from the body of the input text',
      'parameters': {
          'type': 'object',
          'properties': {
              'name': {
                  'type': 'string',
                  'description': 'Name of the person'
              },
              'major': {
                  'type': 'string',
                  'description': 'Major subject.'
              },
              'school': {
                  'type': 'string',
                  'description': 'The university name.'
              },
              'grades': {
                  'type': 'integer',
                  'description': 'GPA of the student.'
              },
              'club': {
                  'type': 'string',
                  'description': 'School club for extracurricular activities. '
              }
              
          }
      }
  }
]

In [2]:
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

In [9]:
import json

student_description = [student_1_description,student_2_description]
for i in student_description:
    response = client.chat.completions.create(
        model = 'pjf-dpo-turbo-35',
        messages = [{'role': 'user', 'content': i}],
        functions = student_custom_functions,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)

{'name': 'David Nguyen', 'major': 'computer science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
{'name': 'Ravi Patel', 'major': 'computer science', 'school': 'University of Michigan', 'grades': 3.7, 'club': 'Chess Club, South Asian Student Association'}


In [10]:
multiple_custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    },
    {
        'name': 'extract_school_info',
        'description': 'Get the school information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the school.'
                },
                'ranking': {
                    'type': 'integer',
                    'description': 'QS world ranking of the school.'
                },
                'country': {
                    'type': 'string',
                    'description': 'Country of the school.'
                },
                'no_of_students': {
                    'type': 'integer',
                    'description': 'Number of students enrolled in the school.'
                }
            }
        }
    }
]

In [11]:
school_1_description = "Stanford University is a private research university located in Stanford, California, United States. It was founded in 1885 by Leland Stanford and his wife, Jane Stanford, in memory of their only child, Leland Stanford Jr. The university is ranked #5 in the world by QS World University Rankings. It has over 17,000 students, including about 7,600 undergraduates and 9,500 graduates23. "

In [13]:
import json

student_description = [student_1_description,school_1_description]
for i in student_description:
    response = client.chat.completions.create(
        model = 'pjf-dpo-turbo-35',
        messages = [{'role': 'user', 'content': i}],
        functions = multiple_custom_functions,
        function_call = 'auto'
    )

    # Loading the response as a JSON object
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)

{'name': 'David Nguyen', 'major': 'computer science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
{'name': 'Stanford University', 'ranking': 5, 'country': 'United States', 'no_of_students': 17000}


TODO:
* execute custom SQL queries. Extract relevant information from text and provide consistent responses for API and SQL commands.